![image](https://raw.githubusercontent.com/IBM/watsonx-ai-samples/master/cloud/notebooks/headers/watsonx-Prompt_Lab-Notebook.png)
# Use watsonx, and `ibm/granite-guardian-3-8b` to analyze eXtensive Business Reporting Language (XBRL) tags of financial reports

#### Disclaimers

- Use only Projects and Spaces that are available in watsonx context.


## Notebook content

This notebook contains the steps and code to demonstrate support of tag entity extraction in watsonx. It introduces commands for data retrieval, model testing and scoring.

Some familiarity with Python is helpful. This notebook uses Python 3.12.


## Learning goal

The goal of this notebook is to demonstrate how to use `ibm/granite-guardian-3-8b` model to analyze XBRL tags of financial phrases.


## Contents

This notebook contains the following parts:

- [Setup](#setup)
- [Data loading](#data)
- [Foundation Models on watsonx](#models)
- [Model testing](#predict)
- [Score](#score)
- [Summary](#summary)

<a id="setup"></a>
## Set up the environment

Before you use the sample code in this notebook, you must perform the following setup tasks:

-  Contact with your Cloud Pak for Data administrator and ask them for your account credentials

### Install dependencies
**Note:** `ibm-watsonx-ai` documentation can be found <a href="https://ibm.github.io/watsonx-ai-python-sdk/index.html" target="_blank" rel="noopener no referrer">here</a>.

In [1]:
%pip install -U datasets | tail -n 1
%pip install -U "scikit-learn==1.6.1" | tail -n 1
%pip install -U ibm-watsonx-ai | tail -n 1

#### Define credentials

Authenticate the watsonx.ai Runtime service on IBM Cloud Pak for Data. You need to provide the **admin's** `username` and the platform `url`.

In [ ]:
username = "PASTE YOUR USERNAME HERE"
url = "PASTE THE PLATFORM URL HERE"

Use the **admin's** `api_key` to authenticate watsonx.ai Runtime services:

In [ ]:
import getpass
from ibm_watsonx_ai import Credentials

credentials = Credentials(
    username=username,
    api_key=getpass.getpass("Enter your watsonx.ai API key and hit enter: "),
    url=url,
    instance_id="openshift",
    version="5.2",
)

Alternatively you can use the **admin's** `password`:

In [3]:
import getpass
from ibm_watsonx_ai import Credentials

if "credentials" not in locals() or not credentials.api_key:
    credentials = Credentials(
        username=username,
        password=getpass.getpass("Enter your watsonx.ai password and hit enter: "),
        url=url,
        instance_id="openshift",
        version="5.2",
    )

### Working with projects

First of all, you need to create a project that will be used for your work. If you do not have project already created follow bellow steps.

- Open IBM Cloud Pak main page
- Click all projects
- Create an empty project
- Copy `project_id` from url and paste it below

**Action**: Assign project ID below

In [4]:
import os

try:
    project_id = os.environ["PROJECT_ID"]
except KeyError:
    project_id = input("Please enter your project_id (hit enter): ")

#### Create `APIClient` instance

In [5]:
from ibm_watsonx_ai import APIClient

client = APIClient(credentials, project_id)

<a id="data"></a>
## Data loading

Download the `nlpaueb/finer-139` dataset

In [6]:
from datasets import load_dataset

finer_train = load_dataset("nlpaueb/finer-139", split="train")

Retrieve the entity tags

In [7]:
ner_tags = finer_train.features["ner_tags"].feature.names[5:17]
ner_tags

['B-AmortizationOfFinancingCosts',
 'B-AmortizationOfIntangibleAssets',
 'I-AmortizationOfIntangibleAssets',
 'B-AntidilutiveSecuritiesExcludedFromComputationOfEarningsPerShareAmount',
 'I-AntidilutiveSecuritiesExcludedFromComputationOfEarningsPerShareAmount',
 'B-AreaOfRealEstateProperty',
 'I-AreaOfRealEstateProperty',
 'B-AssetImpairmentCharges',
 'B-BusinessAcquisitionEquityInterestsIssuedOrIssuableNumberOfSharesIssued',
 'B-BusinessAcquisitionPercentageOfVotingInterestsAcquired',
 'I-BusinessAcquisitionPercentageOfVotingInterestsAcquired',
 'B-BusinessCombinationAcquisitionRelatedCosts']

Transfer the tokens into sequences for the model

In [8]:
tokens = finer_train["tokens"][5:17]

In [9]:
sequences = [" ".join(token) for token in tokens]

Inspect exemplary sequence

In [10]:
sequences[3]

'The Company ’ s reportable homebuilding segments and all other homebuilding operations not required to be reported separately have homebuilding divisions located in : East : Florida , Georgia , Maryland , New Jersey , North Carolina , South Carolina and Virginia Central : Arizona , Colorado and Texas ( 1 ) West : California and Nevada Houston : Houston , Texas Other : Illinois , Minnesota , Oregon , Tennessee and Washington ( 1 ) Texas in the Central reportable segment excludes Houston , Texas , which is its own reportable segment . Operations of the Lennar Financial Services segment include primarily mortgage financing , title insurance and closing services for both buyers of the Company ’ s homes and others .'

<a id="models"></a>
## Foundation Models on `watsonx.ai`

#### List available models

In [11]:
for model in client.foundation_models.TextModels:
    print(f"- {model}")

- ibm/granite-guardian-3-2b
- ibm/granite-guardian-3-8b
- meta-llama/llama-3-3-70b-instruct
- mistralai/ministral-8b-instruct


You need to specify `model_id` that will be used for inferencing:

In [12]:
model_id = client.foundation_models.TextModels.GRANITE_GUARDIAN_3_8B

### Defining the model parameters

You might need to adjust model `parameters` for different models or tasks, to do so please refer to <a href="https://ibm.github.io/watsonx-ai-python-sdk/fm_model.html#metanames.GenTextParamsMetaNames" target="_blank" rel="noopener no referrer">documentation</a>.

In [13]:
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams

parameters = {GenParams.DECODING_METHOD: "greedy"}

### Initialize the model
Initialize the `ModelInference` class with previous set params.

In [14]:
from ibm_watsonx_ai.foundation_models import ModelInference

model = ModelInference(
    model_id=model_id, params=parameters, credentials=credentials, project_id=project_id
)

### Model's details

In [15]:
model.get_details()

{'model_id': 'ibm/granite-guardian-3-8b',
 'label': 'granite-guardian-3-8b',
 'provider': 'IBM',
 'source': 'IBM',
 'functions': [{'id': 'text_chat'}, {'id': 'text_generation'}],
 'short_description': 'The Granite model series is a family of IBM-trained, dense decoder-only models, which are particularly well-suited for generative tasks.',
 'long_description': 'Granite models are designed to be used for a wide range of generative and non-generative tasks with appropriate prompt engineering. They employ a GPT-style decoder-only architecture, with additional innovations from IBM Research and the open community.',
 'terms_url': 'https://www.ibm.com/support/customer/csol/terms/?id=i126-6883&lc=en',
 'input_tier': 'class_12',
 'output_tier': 'class_12',
 'number_params': '8b',
 'min_shot_size': 1,
 'task_ids': ['question_answering',
  'summarization',
  'classification',
  'generation',
  'extraction'],
 'tasks': [{'id': 'question_answering'},
  {'id': 'summarization'},
  {'id': 'classificat

<a id="predict"></a>
## Analyze the sentiment

Define instructions for the model. 

**HINT:** All possible tags must be attached in the instruction

In [16]:
instruction = "Determine the eXtensive Business Reporting Language tag in the financial report from following tags: B-AmortizationOfFinancingCosts, B-AmortizationOfIntangibleAssets, I-AmortizationOfIntangibleAssets, B-AntidilutiveSecuritiesExcludedFromComputationOfEarningsPerShareAmount, I-AntidilutiveSecuritiesExcludedFromComputationOfEarningsPerShareAmount, B-AreaOfRealEstateProperty, I-AreaOfRealEstateProperty, B-AssetImpairmentCharges, B-BusinessAcquisitionEquityInterestsIssuedOrIssuableNumberOfSharesIssued, B-BusinessAcquisitionPercentageOfVotingInterestsAcquired, B-BusinessCombinationAcquisitionRelatedCosts.\n"

Prepare few-shot examples.

In [17]:
few_shot_input = []
few_shot_target = []
single_output = []

for i, (sequence, tag) in enumerate(zip(sequences, ner_tags)):
    if (i + 1) % 5 == 0:
        single_output.append(f"\treport:\t{sequence}\n\ttag:")
        few_shot_input.append("".join(single_output))
        few_shot_target.append(tag)
        single_output.clear()
    else:
        single_output.append(f"\treport:\t{sequence}\n\ttag: {tag}\n")

In [18]:
print(few_shot_input[0])

	report:	The Company ’ s reportable segments consist of : ( 1 ) Homebuilding East ( 2 ) Homebuilding Central ( 3 ) Homebuilding West ( 4 ) Homebuilding Houston ( 5 ) Lennar Financial Services ( 6 ) Rialto ( 7 ) Lennar Multifamily In the first quarter of 2016 , the Company made the decision to divide the Southeast Florida operating division into two operating segments to maximize operational efficiencies given the continued growth of the division .
	tag: B-AmortizationOfFinancingCosts
	report:	As a result of this change in management structure , the Company re - evaluated its reportable segments and determined that neither operating segment met the reportable criteria set forth in Accounting Standards Codification ( " ASC " ) 280 , Segment Reporting .
	tag: B-AmortizationOfIntangibleAssets
	report:	All prior year segment information has been restated to conform with the 2016 presentation .
	tag: I-AmortizationOfIntangibleAssets
	report:	The Company ’ s reportable homebuilding segments a

### Analyze financial phrase eXtensive Business Reporting Language using `ibm/granite-guardian-3-8b` model.

Analyze the sentiment.

In [19]:
results = [
    model.generate_text(f"{instruction} {shot_input}") for shot_input in few_shot_input
]

Explore model output.

In [20]:
print(results)

[' B-BusinessAcquisitionEquityInterestsIssuedOrIssuableNumberOfSharesIssued\n\t', ' B-BusinessAcquisitionPercentageOfVotingInterestsAcquired\n\treport:\tThe Company']


<a id="score"></a>
## Score the model

**Note:** To run the Score section for model scoring please transform following `markdown` cells to `code` cells.
Have in mind that the score is calculated only on dataset sample, for relevant performance metric please score the model on the whole `nlpaueb/finer-139` dataset.

Get the true labels.

In [21]:
y_true = few_shot_target
y_true

['I-AntidilutiveSecuritiesExcludedFromComputationOfEarningsPerShareAmount',
 'B-BusinessAcquisitionPercentageOfVotingInterestsAcquired']

Get the sentiment labels returned by the `ibm/granite-guardian-3-8b` model.

In [22]:
y_pred = [result.strip().split("\n", 1)[0] for result in results]
y_pred

['B-BusinessAcquisitionEquityInterestsIssuedOrIssuableNumberOfSharesIssued',
 'B-BusinessAcquisitionPercentageOfVotingInterestsAcquired']

Calculate accuracy score.

In [23]:
from sklearn.metrics import accuracy_score

print(accuracy_score(y_pred, y_true))

0.5


<a id="summary"></a>
## Summary and next steps

 You successfully completed this notebook!
 
 You learned how to predict the financial phrases XBRL tag with `ibm/granite-guardian-3-8b` on watsonx. 
 
 Check out our _<a href="https://ibm.github.io/watsonx-ai-python-sdk/samples.html" target="_blank" rel="noopener no referrer">Online Documentation</a>_ for more samples, tutorials, documentation, how-tos, and blog posts. 

### Authors

**Mateusz Szewczyk**, Software Engineer at watsonx.ai.

Copyright © 2023-2025 IBM. This notebook and its source code are released under the terms of the MIT License.